In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
url = 'https://m.imdb.com/chart/top/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

req = requests.get(url, headers=header)
bs = BeautifulSoup(req.text, 'html.parser')

In [ ]:
df = pd.DataFrame({ 'Name': [], 'Ranking': [], 'Score': [], 'Review Amount': [], 'Year': [], 'Length': [], 'Rate': [], 'Page Url': [] })

In [ ]:
li_list = bs.find_all('li', { 'class': 'ipc-metadata-list-summary-item' })

for li in li_list:
  h3 = li.find('h3').text
  ranking, title = h3.split('. ', 1)
  score = li.find('span', { 'class':'ipc-rating-star--imdb' })['aria-label'].split(': ')[1]
  reviews_amount = li.find('span', { 'class': 'ipc-rating-star--voteCount' }).text
  metadata = li.find('div', { 'class': 'cli-title-metadata' })
  year = metadata.find_all('span')[0].text
  length = metadata.find_all('span')[1].text
  rate = 'Not Rated'
  if len(metadata.find_all('span')) > 2:
    rate = metadata.find_all('span')[2].text
  
  query = li.find('a', { 'class': 'ipc-title-link-wrapper' })['href'].split('/?')[0]
  page_url = 'https://m.imdb.com' + query

  df.loc[len(df.index)] = [title, ranking, score, reviews_amount, year, length, rate, page_url]


In [ ]:
for i, row in df.iterrows():
  page_url = row['Page Url']
  req = requests.get(page_url, headers=header)
  bs = BeautifulSoup(req.text, 'html.parser')

  container = bs.find('div', { 'class': 'iOESUA' })
  uls = container.find_all('ul', { 'class': 'ipc-inline-list--show-dividers' })
  directors = uls[0].find_all('a', { 'class': 'ipc-metadata-list-item__list-content-item' })
  writers = uls[1].find_all('a', { 'class': 'ipc-metadata-list-item__list-content-item' })
  stars = uls[2].find_all('a', { 'class': 'ipc-metadata-list-item__list-content-item' })

  director = ''
  for director_tag in directors:
    director += director_tag.text + ','
  director = director[:-1]

  writer = ''
  for writer_tag in writers:
    writer += writer_tag.text + ','
  writer = writer[:-1]

  star = ''
  for star_tag in stars:
    star += star_tag.text + ','
  star = star[:-1]

  df.loc[i, 'Directors'] = director
  df.loc[i, 'Writers'] = writer
  df.loc[i, 'Stars'] = star

  print(i, row['Name'])

  time.sleep(0.5)

In [ ]:
df.to_excel('movies.xlsx')